Script to view what the templates would look like after rendering.

In [10]:
from pathlib import Path

import chevron
from rich import print

from bids import BIDSLayout

In [11]:
def root_dir() -> Path:
    return Path().absolute().parents[1]


def highlight_missing_tags(text: str) -> str:
    """Highlights missing info in red."""
    text = text.replace("{{", "[red]{{")
    text = text.replace("}}", "}}[/red]")
    return text


def print_to_screen(layout, template_name, suffix, keep) -> None:
    """Get files for a certain suffix and render their method section."""
    files = layout.get(suffix=suffix)

    if len(files) < 1:
        return

    metadata = files[0].get_metadata()
    metadata["suffix"] = suffix

    render_template(template_name, metadata, keep)


def render_template(template_name, metadata, keep) -> None:
    """Render a specific template"""
    templates_dir = root_dir() / "templates"

    partials_path = root_dir() / "partials"

    with open(templates_dir / template_name, "r") as template:
        args = {
            "template": template,
            "data": metadata,
            "partials_path": partials_path,
            "keep": keep,
        }

        text = chevron.render(**args)

        print(highlight_missing_tags(text))

def render_method_section_bids_dataset(dataset, keep):
    data_dir = root_dir() / "demos" / "data" / "bids-examples"

    data_path = data_dir / dataset

    layout = BIDSLayout(data_path)

    # %%
    print(f"\n[bold]# MRI[/bold]")

    files = layout.get(suffix="T1w")
    if len(files) > 1:
        metadata = files[0].get_metadata()
        render_template("institution.mustache", metadata, keep)
        render_template("mri_scanner_info.mustache", metadata, keep)

    suffix_template = {
        "T1w": "anat",
        "bold": "func",
        "dwi": "dwi",
        "perf": "perf",
    }

    for suffix, template in suffix_template.items():
        print(f"\n[bold]## {suffix.upper()}[/bold]")
        print_to_screen(layout, f"{template}.mustache", suffix, keep)

    # %%
    print(f"\n[bold]# PET[/bold]")

    print(f"\n[bold]## {'pet'.upper()}[/bold]")
    print_to_screen(layout, "pet.mustache", suffix, keep)

    # %%
    print(f"\n[bold]# MEEG[/bold]")

    suffix_template = {
        "eeg": "meeg",
        "meg": "meeg",
    }

    for suffix, template in suffix_template.items():
        print(f"\n[bold]## {suffix.upper()}[/bold]")
        print_to_screen(layout, f"{template}.mustache", suffix, keep)

In [12]:
# Toggling "keep" to True will render the empty variables in RED
# Only work with the version of chevron from github and not pypi
keep = True

# bids examples dataset to work with
# see in  "demos/data/bids-examples"
dataset = "ds000117"

render_method_section_bids_dataset(dataset=dataset, keep=keep)

# MRI

The data acquisition was performed in the {{ InstitutionName }}, {{ InstitutionalDepartmentName }}, {{ 
InstitutionAddress }}.

MRI data were acquired on a {{ MagneticFieldStrength }} Tesla MRI scanner using a {{ ManufacturersModelName }} 
system from {{ Manufacturer }} with serial number {{ DeviceSerialNumber }}.
The software version was {{ SoftwareVersions }}.

## T1W

{{ nb_runs }} of {{ scan_type }} {{ PulseSequenceType }} {{ seqs }} {{ variants }} {{ multi_echo }} structural MRI 
data were collected (repetition time, TR= {{ tr }} ms; flip angle, FA= {{ FlipAngle }}°; echo time, TE= {{ 
echo_time }} ms; field of view FOV= {{ fov }} mm; matrix size= {{ matrix_size }}; voxel size= {{ voxel_size }} mm; 
{{ nb_slices }} slices).

...

## PET

# MEEG

## EEG

## MEG

Recordings were done using a ElektaVectorview system from ElektaNeuromag with serial number {{ DeviceSerialNumber 
}}.
The software version was {{ SoftwareVersions }}.

Power line and sampling frequencies were 50 Hz and 1100 Hz, respectively.

Data was collected with {{ ChannelCount }} meg channels, 2 EOG channels,
1 ECG channels and 0 EMG channels recorded simultaneously.

Subjects viewed stimuli on a screen during six, 7.5 minute runs. The stimuli were photographs of either a face or a
scrambled face, and appeared for a random duration between 800 and 1,000 ms. Subjects were instructed to fixate 
centrally throughout the experiment. To ensure attention to each stimulus, participants were asked to press one of 
two keys with either their left or right index finger (assignment counter-balanced across participants). Their 
key-press was based on how symmetric they regarded each image: pressing one or the other key depending whether they
thought the image was 'more' or 'less symmetric' than average. See publication for more detailed information.
Participants were specifically instructed to: {{ Instructions }}